In [ ]:
path = "/content/drive/MyDrive/BracU/Summer-21/AAI/Project"
from google.colab import drive
drive.mount("/content/drive")
%cd $path

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

# Dataset Processing
### The _DSEBD_ dataset is taken from the [https://www.kaggle.com/mahmudulhaque/dsebd](https://www.kaggle.com/mahmudulhaque/dsebd)
### It is divided into several json files. Each json files contains yearly trading info of companies registered to Dhaka Stock Exchange

* The json files are loaded into pandas dataframe
* Dataframes are merged into single frame
* Entire frame is into csv format
* A subset of dataframe is created with smaller number of attributes 
* Data for each companies are stored into separate csv files.


In [5]:
meta_json = 'securities.json'

info_frame = pd.read_json('dsebd/'+meta_json)
info_frame.to_csv('meta_data.csv', encoding='utf-8')
info_frame.describe()

,trading_code,sector,instrument_type
count,589,589,589
unique,589,22,5
top,NAVANACNG,Treasury Bond,Equity
freq,1,221,321


In [6]:
info_frame['instrument_type'].unique()

array(['Equity', 'Mutual Fund', 'Corporate Bond', 'Debenture',
       'Treasury Bond'], dtype=object)

In [9]:
info_frame.sample(5)

,trading_code,sector,instrument_type
82,BXPHARMA,Pharmaceuticals & Chemicals,Equity
275,POWERGRID,Fuel & Power,Equity
369,T10Y0617,Treasury Bond,Treasury Bond
260,PDL,Textile,Equity
389,T10Y0919,Treasury Bond,Treasury Bond


In [10]:
prices_jsons  = ['prices_2008.json','prices_2009.json','prices_2010.json',
              'prices_2011.json','prices_2012.json','prices_2013.json',
             'prices_2014.json','prices_2015.json','prices_2016.json',
             'prices_2017.json','prices_2018.json','prices_2019.json','prices_2020.json']

frame_list = []
for i in range(len(prices_jsons)):
    temp_frame = pd.read_json('dsebd/'+prices_jsons[i])
    #print(temp_frame.head())
    frame_list.append(temp_frame)

In [5]:
dse_frame = pd.concat(frame_list)
#dse_frame = dse_frame.set_index('date')
dse_frame.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1575134 entries, 0 to 116742
Data columns (total 11 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   date                      1575134 non-null  datetime64[ns]
 1   trading_code              1575134 non-null  object        
 2   last_traded_price         1575134 non-null  float64       
 3   high                      1575134 non-null  float64       
 4   low                       1575134 non-null  float64       
 5   opening_price             1575134 non-null  float64       
 6   closing_price             1575134 non-null  float64       
 7   yesterdays_closing_price  1575134 non-null  float64       
 8   trade                     1575134 non-null  int64         
 9   value_mn                  1575134 non-null  float64       
 10  volume                    1575134 non-null  int64         
dtypes: datetime64[ns](1), float64(7), int64(2), object(

In [14]:
dse_frame.head()

,date,trading_code,last_traded_price,high,low,opening_price,closing_price,yesterdays_closing_price,trade,value_mn,volume
0,2008-12-30,1STBSRS,823.00,840.00,819.0,825.0,826.25,825.75,86,4.1475,5000
1,2008-12-28,1STBSRS,817.00,829.00,802.0,802.0,825.75,790.00,59,2.7838,3400
2,2008-12-24,1STBSRS,789.00,809.75,786.0,790.0,790.00,785.75,74,3.5270,4450
3,2008-12-23,1STBSRS,785.00,798.50,785.0,785.0,785.75,782.50,38,2.0541,2600
4,2008-12-22,1STBSRS,778.25,794.50,775.5,787.0,782.50,797.75,19,0.7825,1000


In [15]:
dse_frame.to_csv('dsebd.csv', encoding='utf-8', index=False)

In [7]:
dse_frame = pd.read_csv('dsebd.csv')
#dse_frame.head()

**create csv for each company**

In [8]:
company_list = info_frame['trading_code']
sub_frame = dse_frame.loc[:, ['date', 'trading_code', 'high', 'low', 'opening_price', 'closing_price', 
'yesterdays_closing_price', 'volume']]

sub_frame = sub_frame[~(sub_frame["opening_price"] == 0) | (sub_frame["closing_price"] == 0) | (sub_frame["high"] == 0)
    | (sub_frame["low"] == 0) | (sub_frame["yesterdays_closing_price"] == 0)]

sub_frame = sub_frame.sort_values(["date"], ascending=True)
sub_frame['date'] = pd.to_datetime(sub_frame['date'])
#sub_frame = sub_frame.set_index(["date"], inplace=True)

for i in range(len(info_frame)):
    company_code = company_list[i].strip()
    company_data = sub_frame[sub_frame['trading_code']==company_code]
    company_data.rename(columns=({ 'opening_price': 'open', 'closing_price': 'close',
    'yesterdays_closing_price':'prev_closing_price'}), inplace=True,)
   
    company_data.to_csv('./company_data/{company}.csv'.format(company=company_code),
    columns=['date', 'open', 'high', 'low', 'close', 'volume', 'prev_closing_price'],index=False)
    print(company_code, 'data saved as CSV')

C:\Users\raihan\Anaconda3\lib\site-packages\pandas\core\frame.py:4293: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


ABBANK data saved as CSV
ACI data saved as CSV
ACFL data saved as CSV
AAMRATECH data saved as CSV
1JANATAMF data saved as CSV
AAMRANET data saved as CSV
ACMELAB data saved as CSV
AFCAGRO data saved as CSV
ACTIVEFINE data saved as CSV
ADNTEL data saved as CSV
ACIFORMULA data saved as CSV
ADVENT data saved as CSV
AL-HAJTEX data saved as CSV
AGRANINS data saved as CSV
AIBL1STIMF data saved as CSV
ALARABANK data saved as CSV
ABB1STMF data saved as CSV
1STPRIMFMF data saved as CSV
AMCL(PRAN) data saved as CSV
AMBEEPHA data saved as CSV
AGNISYSL data saved as CSV
AFTABAUTO data saved as CSV
AIL data saved as CSV
ALLTEX data saved as CSV
APEXSPINN data saved as CSV
APEXFOODS data saved as CSV
ALIF data saved as CSV
APEXTANRY data saved as CSV
AMANFEED data saved as CSV
APOLOISPAT data saved as CSV
ARAMIT data saved as CSV
ARGONDENIM data saved as CSV
ASIAPACINS data saved as CSV
ANWARGALV data saved as CSV
ANLIMAYARN data saved as CSV
APSCLBOND data saved as CSV
BANGAS data saved as CSV
BANKA

**Data Pre-processing Done!**